In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm, truncnorm, bernoulli
import cloudpickle as pickle
import matplotlib.pyplot as plt
import torch
import warnings
warnings.filterwarnings("ignore")

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all' #last_expr

In [2]:
def simu_environment_component(N, MEAN_E0, MEAN_E1, SIGMA_E0, SIGMA_E1, MIN_E, MAX_E):
    dc_D0 = np.round(norm.rvs(loc=MEAN_E0, scale=SIGMA_E0, size=N), 3).clip(MIN_E, MAX_E)
    dc_D1 = np.round(norm.rvs(loc=MEAN_E1, scale=SIGMA_E1, size=N), 3).clip(MIN_E, MAX_E)
    return dc_D0, dc_D1

def simu_age_component(N, MEAN_A0, MEAN_A1, SIGMA_A0, SIGMA_A1, MIN_A, MAX_A):
    ac_a0 = np.round(norm.rvs(loc=MEAN_A0, scale=SIGMA_A0, size=N),3).clip(MIN_A, MAX_A)
    ac_a1 = np.round(norm.rvs(loc=MEAN_A1, scale=SIGMA_A1, size=N),3).clip(MIN_A, MAX_A)
    return ac_a0, ac_a1

def simu_sex_component(N, MEAN_S0, MEAN_S1, SIGMA_S0, SIGMA_S1, MIN_S, MAX_S):
    sc_s0 = np.round(norm.rvs(loc=MEAN_S0, scale=SIGMA_S0, size=N), 3).clip(MIN_S, MAX_S)
    sc_s1 = np.round(norm.rvs(loc=MEAN_S1, scale=SIGMA_S1, size=N), 3).clip(MIN_S, MAX_S)
    return sc_s0, sc_s1

def simu_noise_component(N, MEAN_N, SIGMA_N, MIN_N, MAX_N):
    #np.random.seed(0)
    #nc_n = np.round(norm.rvs(loc=MEAN_N, scale=SIGMA_N, size=N*2), 3).clip(MIN_N, MAX_N)
    #pickle.dump(nc_n, open('nc_n.pkl', 'wb'))
    nc_n = pickle.load(open('nc_n.pkl', 'rb'))
    return nc_n

def simi_metabolic_component(N, MEAN_E0, MEAN_E1, SIGMA_E0, SIGMA_E1, MIN_E, MAX_E, MEAN_A0, MEAN_A1, SIGMA_A0, SIGMA_A1, MIN_A, MAX_A, MEAN_S0, MEAN_S1, SIGMA_S0, SIGMA_S1, MIN_S, MAX_S,MEAN_N, SIGMA_N, MIN_N, MAX_N):
    dc_D0, dc_D1 = simu_environment_component(N, MEAN_E0, MEAN_E1, SIGMA_E0, SIGMA_E1, MIN_E, MAX_E)
    ac_a0, ac_a1 = simu_age_component(N, MEAN_A0, MEAN_A1, SIGMA_A0, SIGMA_A1, MIN_A, MAX_A)
    sc_s0, sc_s1 = simu_sex_component(N, MEAN_S0, MEAN_S1, SIGMA_S0, SIGMA_S1, MIN_S, MAX_S)
    nc_n = simu_noise_component(N, MEAN_N, SIGMA_N, MIN_N, MAX_N)
    simudata = pd.DataFrame([np.hstack((dc_D0, dc_D1)), np.hstack((ac_a0, ac_a1)), np.hstack((sc_s0, sc_s1)), np.hstack((nc_n)),np.ones(N*2)], index=['Environment', 'Age', 'Sex','Noise','1'], columns=range(N*2)).T
    #simudata['Noise'] = np.round(norm.rvs(MEAN_U, SIGMA_U, size=2*N), 3)
    #Group=0 : control  Group=1 : case
    simudata['Group'] = np.hstack(([0 for i in range(N)], [1 for i in range(N)]))
    return simudata

def simu_weight(N_Species, MEAN_WE, MEAN_WA, MEAN_WS,MEAN_WN, MEAN_WD, rsize=50):
    simudata = []
    Ws = []
    for i in range(N_Species):
        w = np.random.choice(['E', 'A', 'S','N', 'D'], size=rsize, p=[MEAN_WE, MEAN_WA, MEAN_WS, MEAN_WN,MEAN_WD])
        w = np.round([(w==i).mean() for i in ['E', 'A', 'S','N', 'D']], 3)
        w = list(w)
        w. append(np.random.random( ))
        Ws.append(w)
    return pd.DataFrame(Ws, index=range(N_Species), columns=['E', 'A', 'S','N', 'D','1'])

def cal_abundance(metabolic, weights, windex):
    abundance = pd.DataFrame(np.dot(metabolic[['Environment', 'Age', 'Sex','Noise','Group','1']].values, weights.values.T), index=metabolic.index, 
                             columns=['W'+str(windex)+'X'+str(i) for i in weights.index])
    metabolic = pd.concat([metabolic, abundance], axis=1, sort=False)
    return metabolic

def simu_main_meta(config,Liner):
    metabolic = simi_metabolic_component(config['N'], config['MEAN_E0'], config['MEAN_E1'], config['SIGMA_E0'], config['SIGMA_E1'], config['MIN_E'], config['MAX_E'],
                                         config['MEAN_A0'], config['MEAN_A1'], config['SIGMA_A0'], config['SIGMA_A1'], config['MIN_A'], config['MAX_A'],
                                         config['MEAN_S0'], config['MEAN_S1'], config['SIGMA_S0'], config['SIGMA_S1'], config['MIN_S'], config['MAX_S'],
                                         config['MEAN_N'], config['SIGMA_N'], config['MIN_N'], config['MAX_N'])
    if Liner==True:
        metabolic['Environment'] =config['LE_E'] *metabolic['Group'] + metabolic['Environment']  
        metabolic['Age']=config['LE_A'] * metabolic['Group'] + metabolic['Age']
        metabolic['Sex']=config['LE_S'] * metabolic['Group'] + metabolic['Sex']
    return metabolic

def simu_main_microbiome(metabolic,config):
    windex = 1
    for Ws in config['Weights_proportion']:
        weights = simu_weight(Ws['N_Species'], Ws['MEAN_WE'], Ws['MEAN_WA'], Ws['MEAN_WS'], Ws['MEAN_WN'],Ws['MEAN_WD'])
        metabolic = cal_abundance(metabolic, weights, windex)
        windex += 1
    return metabolic

In [3]:
def validation_score(DiffTax,PSMDiffTax,TrueDiffTax,OtherTax,PSMOtherTax,TrueOtherTax):
    TP1=len(DiffTax.intersection(TrueDiffTax))
    FP1=len(DiffTax)-TP1
    TN1=len(OtherTax.intersection(TrueOtherTax))
    FN1=len(OtherTax)-TN1
    TP2=len(PSMDiffTax.intersection(TrueDiffTax))
    FP2=len(PSMDiffTax)-TP2
    TN2=len(PSMOtherTax.intersection(TrueOtherTax))
    FN2=len(PSMOtherTax)-TN2
    accuracy1=(TP1+TN1)/(TP1+FN1+FP1+TN1)
    precision1=TP1/(TP1+FP1)
    recall1=TP1/(TP1+FN1)
    F11=2*precision1*recall1/(precision1+recall1)
    accuracy2=(TP2+TN2)/(TP2+FN2+FP2+TN2)
    if (TP2+FP2)==0:
        precision2=0
    else:
        precision2=TP2/(TP2+FP2)
    recall2=TP2/(TP2+FN2)
    if (precision2+recall2)==0:
        F12=None
    else:
        F12=2*precision2*recall2/(precision2+recall2)
    score=pd.DataFrame(data=None,columns=["Accuracy","Precision","Recall","F1","psm-Accuracy","psm-Precision","psm-Recall","psm-F1"])
    score.loc[len(score.index)] = [accuracy1,precision1,recall1,F11,accuracy2,precision2,recall2,F12]
    return score

In [4]:
def save_files(pi,seed,config,metadata,microbiome,result):
    s=str(seed)
    with open('../simuData/'+pi+'/config.txt','w') as f:f.write(str(config))
    metadata.to_csv( '../simuData/'+pi+'/'+s+'/metadata.csv')
    microbiome.to_csv( '../simuData/'+pi+'/'+s+'/microbiome.csv')
    result.to_csv( '../simuData/'+pi+'/'+s+'/PSMresult.csv')

In [5]:
%run miMatch.py
psm = miMatch()

# 3. Simulated metabolic background with differential trial effects
## · Disease related to background  and mean difference=0.1(Controls=N[0.45,0.2],Cases=N[0.55,0.2])

## 3.1 config(LE)=0.02

In [6]:
config_LE_LA_LS = {
    'N':100,
    ### Environment-drivers
    'MEAN_E0':0.45,
    'MEAN_E1':0.55,
    'SIGMA_E0':0.2,
    'SIGMA_E1':0.2,
    'MIN_E':0,
    'MAX_E':1,
    ### Age
    'MEAN_A0':0.45,
    'MEAN_A1':0.55,
    'SIGMA_A0':0.2,
    'SIGMA_A1':0.2,
    'MIN_A':0,
    'MAX_A':1,
    ### Sex
    'MEAN_S0':0.45,
    'MEAN_S1':0.55,
    'SIGMA_S0':0.2,
    'SIGMA_S1':0.2,
    'MIN_S':0,
    'MAX_S':1,
   ### Noise
    'MEAN_N':0.5,
    'SIGMA_N':0.2,
    'MIN_N':0,
    'MAX_N':1,
    'Weights_proportion':[
        { ### 1: non-differetial taxa
        'N_Species':50,
        'MEAN_WE':0.133,
        'MEAN_WA':0.133,
        'MEAN_WS':0.133,
        'MEAN_WN':0.6,
        'MEAN_WD':0.001,
        },
        { ### 2: differetial taxa
        'N_Species':50,
        'MEAN_WE':0.1,
        'MEAN_WA':0.1,
        'MEAN_WS':0.1,
        'MEAN_WN':0.6,
        'MEAN_WD':0.1,
        },
    ],
    'LE_E':0.02,
    'LE_A':0.02,
    'LE_S':0.02,
}


In [7]:
diff=pd.DataFrame()
for seed in range(0,50):
    metadata= simu_main_meta(config_LE_LA_LS,Liner=True)
    microbiome=simu_main_microbiome(metadata,config_LE_LA_LS)
    microbiome=microbiome.iloc[:,6:]
    microbiome['Group'] =metadata['Group']
    metadata.index=['s'+str(i) for i in metadata.index]
    microbiome.index=['s'+str(i) for i in microbiome.index]
    pi='config_LE_LA_LS'
    params = [('output', 'output_dir', '../simuData/'+pi+'/'+str(seed)+'/'), ('psm', 'caliper', str(0.05)), ('psm', 'ratio', str(1))]
    sample_size, match_drop_unmatched, match, pairs, sum_matched, balance_stats = run_simu_match(data=metadata, target='Group', params=params,features=['Environment', 'Age', 'Sex'],caliper=0.05,ratio=1)
    
    ### difference microbiome
    result = pd.DataFrame()
    res = diff_by_rank_sum(microbiome, target='Group', features=[i for i in microbiome.columns if i!='Group'])
    res.columns = [i+'(raw)' for i in res.columns]
    result = pd.concat([result, res], axis=1, sort=False)
    res = diff_by_signed_rank(microbiome, pairs, features=[i for i in microbiome.columns if i!='Group'])
    res.columns = [i+'(PSM)' for i in res.columns]
    result = pd.concat([result, res], axis=1, sort=False)
    result = result.sort_values(['fdr(PSM)'])
    TrueDiffTax=microbiome.columns[50:100]
    TrueOtherTax=microbiome.columns[0:50]
    DiffTax=result.index[result['p-value(raw)']<0.05]
    OtherTax=microbiome.columns[0:100].difference(DiffTax)
    PSMDiffTax=result.index[result['p-value(PSM)']<0.05]
    PSMOtherTax=microbiome.columns[0:100].difference(PSMDiffTax)
    score=validation_score(DiffTax,PSMDiffTax,TrueDiffTax,OtherTax,PSMOtherTax,TrueOtherTax)
    diff=pd.concat([diff,score])
    save_files(pi,seed,config_LE_LA_LS,metadata,microbiome,result)
diff.index=['s'+str(i) for i in range(0,50)]
diff.to_csv('../simuData/LE_LA_LS_changed_1.csv')
diff

,Accuracy,Precision,Recall,F1,psm-Accuracy,psm-Precision,psm-Recall,psm-F1
s0,0.60,0.555556,1.00,0.714286,0.90,1.000000,0.80,0.888889
s1,0.56,0.531915,1.00,0.694444,0.92,1.000000,0.84,0.913043
s2,0.89,0.819672,1.00,0.900901,0.99,1.000000,0.98,0.989899
s3,0.54,0.520833,1.00,0.684932,0.88,1.000000,0.76,0.863636
s4,0.59,0.549451,1.00,0.709220,0.81,1.000000,0.62,0.765432
s5,0.72,0.644737,0.98,0.777778,0.94,1.000000,0.88,0.936170
s6,0.56,0.531915,1.00,0.694444,0.94,1.000000,0.88,0.936170
s7,0.58,0.543478,1.00,0.704225,0.96,1.000000,0.92,0.958333
s8,0.60,0.555556,1.00,0.714286,0.98,1.000000,0.96,0.979592
s9,0.59,0.549451,1.00,0.709220,0.99,1.000000,0.98,0.989899


In [8]:
diff.mean(axis=0)

Accuracy         0.640200
Precision        0.592182
Recall           0.996400
F1               0.739500
psm-Accuracy     0.899400
psm-Precision    0.954907
psm-Recall       0.855600
psm-F1           0.895607
dtype: float64

## 3.2 config(LE)=0.04

In [10]:
config_LE_LA_LS_2 = {
    'N':100,
    ### Environment-drivers
    'MEAN_E0':0.45,
    'MEAN_E1':0.55,
    'SIGMA_E0':0.2,
    'SIGMA_E1':0.2,
    'MIN_E':0,
    'MAX_E':1,
    ### Age
    'MEAN_A0':0.45,
    'MEAN_A1':0.55,
    'SIGMA_A0':0.2,
    'SIGMA_A1':0.2,
    'MIN_A':0,
    'MAX_A':1,
    ### Sex
    'MEAN_S0':0.45,
    'MEAN_S1':0.55,
    'SIGMA_S0':0.2,
    'SIGMA_S1':0.2,
    'MIN_S':0,
    'MAX_S':1,
   ### Noise
    'MEAN_N':0.5,
    'SIGMA_N':0.2,
    'MIN_N':0,
    'MAX_N':1,
    'Weights_proportion':[
        { ### 1: non-differetial taxa
        'N_Species':50,
        'MEAN_WE':0.133,
        'MEAN_WA':0.133,
        'MEAN_WS':0.133,
        'MEAN_WN':0.6,
        'MEAN_WD':0.001,
        },
        { ### 2: differetial taxa
        'N_Species':50,
        'MEAN_WE':0.1,
        'MEAN_WA':0.1,
        'MEAN_WS':0.1,
        'MEAN_WN':0.6,
        'MEAN_WD':0.1,
        },
    ],
    'LE_E':0.04,
    'LE_A':0.04,
    'LE_S':0.04,
}


In [11]:
diff=pd.DataFrame()
for seed in range(0,50):
    metadata= simu_main_meta(config_LE_LA_LS_2,Liner=True)
    microbiome=simu_main_microbiome(metadata,config_LE_LA_LS_2)
    microbiome=microbiome.iloc[:,6:]
    microbiome['Group'] =metadata['Group']
    metadata.index=['s'+str(i) for i in metadata.index]
    microbiome.index=['s'+str(i) for i in microbiome.index]
    pi='config_LE_LA_LS_2'
    params = [('output', 'output_dir', '../simuData/'+pi+'/'+str(seed)+'/'), ('psm', 'caliper', str(0.05)), ('psm', 'ratio', str(1))]
    sample_size, match_drop_unmatched, match, pairs, sum_matched, balance_stats = run_simu_match(data=metadata, target='Group', params=params,features=['Environment', 'Age', 'Sex'],caliper=0.05,ratio=1)
    
    ### difference microbiome
    result = pd.DataFrame()
    res = diff_by_rank_sum(microbiome, target='Group', features=[i for i in microbiome.columns if i!='Group'])
    res.columns = [i+'(raw)' for i in res.columns]
    result = pd.concat([result, res], axis=1, sort=False)
    res = diff_by_signed_rank(microbiome, pairs, features=[i for i in microbiome.columns if i!='Group'])
    res.columns = [i+'(PSM)' for i in res.columns]
    result = pd.concat([result, res], axis=1, sort=False)
    result = result.sort_values(['fdr(PSM)'])
    TrueDiffTax=microbiome.columns[50:100]
    TrueOtherTax=microbiome.columns[0:50]
    DiffTax=result.index[result['p-value(raw)']<0.05]
    OtherTax=microbiome.columns[0:100].difference(DiffTax)
    PSMDiffTax=result.index[result['p-value(PSM)']<0.05]
    PSMOtherTax=microbiome.columns[0:100].difference(PSMDiffTax)
    score=validation_score(DiffTax,PSMDiffTax,TrueDiffTax,OtherTax,PSMOtherTax,TrueOtherTax)
    diff=pd.concat([diff,score])
    save_files(pi,seed,config_LE_LA_LS_2,metadata,microbiome,result)
diff.index=['s'+str(i) for i in range(0,50)]
diff.to_csv('../simuData/LE_LA_LS_changed_2.csv')
diff

,Accuracy,Precision,Recall,F1,psm-Accuracy,psm-Precision,psm-Recall,psm-F1
s0,0.55,0.526316,1.00,0.689655,0.89,1.000000,0.78,0.876404
s1,0.54,0.520833,1.00,0.684932,1.00,1.000000,1.00,1.000000
s2,0.59,0.549451,1.00,0.709220,0.98,0.980000,0.98,0.980000
s3,0.54,0.520833,1.00,0.684932,0.83,1.000000,0.66,0.795181
s4,0.58,0.543478,1.00,0.704225,0.94,1.000000,0.88,0.936170
s5,0.59,0.549451,1.00,0.709220,0.96,0.979167,0.94,0.959184
s6,0.57,0.537634,1.00,0.699301,0.93,1.000000,0.86,0.924731
s7,0.65,0.588235,1.00,0.740741,0.99,1.000000,0.98,0.989899
s8,0.55,0.526316,1.00,0.689655,0.99,0.980392,1.00,0.990099
s9,0.53,0.515789,0.98,0.675862,0.90,1.000000,0.80,0.888889


In [12]:
diff.mean(axis=0)

Accuracy         0.567200
Precision        0.537664
Recall           0.996800
F1               0.698041
psm-Accuracy     0.923800
psm-Precision    0.967680
psm-Recall       0.899600
psm-F1           0.925505
dtype: float64

## 3.3 config(LE)=0.06

In [13]:
config_LE_LA_LS_3 = {
    'N':100,
    ### Environment-drivers
    'MEAN_E0':0.45,
    'MEAN_E1':0.55,
    'SIGMA_E0':0.2,
    'SIGMA_E1':0.2,
    'MIN_E':0,
    'MAX_E':1,
    ### Age
    'MEAN_A0':0.45,
    'MEAN_A1':0.55,
    'SIGMA_A0':0.2,
    'SIGMA_A1':0.2,
    'MIN_A':0,
    'MAX_A':1,
    ### Sex
    'MEAN_S0':0.45,
    'MEAN_S1':0.55,
    'SIGMA_S0':0.2,
    'SIGMA_S1':0.2,
    'MIN_S':0,
    'MAX_S':1,
   ### Noise
    'MEAN_N':0.5,
    'SIGMA_N':0.2,
    'MIN_N':0,
    'MAX_N':1,
    'Weights_proportion':[
        { ### 1: non-differetial taxa
        'N_Species':50,
        'MEAN_WE':0.133,
        'MEAN_WA':0.133,
        'MEAN_WS':0.133,
        'MEAN_WN':0.6,
        'MEAN_WD':0.001,
        },
        { ### 2: differetial taxa
        'N_Species':50,
        'MEAN_WE':0.1,
        'MEAN_WA':0.1,
        'MEAN_WS':0.1,
        'MEAN_WN':0.6,
        'MEAN_WD':0.1,
        },
    ],
    'LE_E':0.06,
    'LE_A':0.06,
    'LE_S':0.06,
}


In [14]:
diff=pd.DataFrame()
for seed in range(0,50):
    metadata= simu_main_meta(config_LE_LA_LS_3,Liner=True)
    microbiome=simu_main_microbiome(metadata,config_LE_LA_LS_3)
    microbiome=microbiome.iloc[:,6:]
    microbiome['Group'] =metadata['Group']
    metadata.index=['s'+str(i) for i in metadata.index]
    microbiome.index=['s'+str(i) for i in microbiome.index]
    pi='config_LE_LA_LS_3'
    params = [('output', 'output_dir', '../simuData/'+pi+'/'+str(seed)+'/'), ('psm', 'caliper', str(0.05)), ('psm', 'ratio', str(1))]
    sample_size, match_drop_unmatched, match, pairs, sum_matched, balance_stats = run_simu_match(data=metadata, target='Group', params=params,features=['Environment', 'Age', 'Sex'],caliper=0.05,ratio=1)
    
    ### difference microbiome
    result = pd.DataFrame()
    res = diff_by_rank_sum(microbiome, target='Group', features=[i for i in microbiome.columns if i!='Group'])
    res.columns = [i+'(raw)' for i in res.columns]
    result = pd.concat([result, res], axis=1, sort=False)
    res = diff_by_signed_rank(microbiome, pairs, features=[i for i in microbiome.columns if i!='Group'])
    res.columns = [i+'(PSM)' for i in res.columns]
    result = pd.concat([result, res], axis=1, sort=False)
    result = result.sort_values(['fdr(PSM)'])
    TrueDiffTax=microbiome.columns[50:100]
    TrueOtherTax=microbiome.columns[0:50]
    DiffTax=result.index[result['p-value(raw)']<0.05]
    OtherTax=microbiome.columns[0:100].difference(DiffTax)
    PSMDiffTax=result.index[result['p-value(PSM)']<0.05]
    PSMOtherTax=microbiome.columns[0:100].difference(PSMDiffTax)
    score=validation_score(DiffTax,PSMDiffTax,TrueDiffTax,OtherTax,PSMOtherTax,TrueOtherTax)
    diff=pd.concat([diff,score])
    save_files(pi,seed,config_LE_LA_LS_3,metadata,microbiome,result)
diff.index=['s'+str(i) for i in range(0,50)]
diff.to_csv('../simuData/LE_LA_LS_changed_3.csv')
diff

,Accuracy,Precision,Recall,F1,psm-Accuracy,psm-Precision,psm-Recall,psm-F1
s0,0.51,0.505051,1.00,0.671141,0.50,0.500000,1.00,0.666667
s1,0.51,0.505051,1.00,0.671141,0.84,1.000000,0.68,0.809524
s2,0.53,0.515464,1.00,0.680272,0.79,1.000000,0.58,0.734177
s3,0.51,0.505155,0.98,0.666667,0.86,1.000000,0.72,0.837209
s4,0.54,0.520833,1.00,0.684932,0.97,1.000000,0.94,0.969072
s5,0.52,0.510204,1.00,0.675676,0.88,1.000000,0.76,0.863636
s6,0.56,0.531915,1.00,0.694444,0.92,0.862069,1.00,0.925926
s7,0.51,0.505051,1.00,0.671141,0.94,0.892857,1.00,0.943396
s8,0.59,0.549451,1.00,0.709220,0.96,0.925926,1.00,0.961538
s9,0.53,0.515464,1.00,0.680272,0.98,1.000000,0.96,0.979592


In [15]:
diff.mean(axis=0)

Accuracy         0.527400
Precision        0.514530
Recall           0.999200
F1               0.679139
psm-Accuracy     0.879200
psm-Precision    0.935466
psm-Recall       0.869200
psm-F1           0.886371
dtype: float64

## 3.4 config(LE)=0.08

In [16]:
config_LE_LA_LS_4 = {
    'N':100,
    ### Environment-drivers
    'MEAN_E0':0.45,
    'MEAN_E1':0.55,
    'SIGMA_E0':0.2,
    'SIGMA_E1':0.2,
    'MIN_E':0,
    'MAX_E':1,
    ### Age
    'MEAN_A0':0.45,
    'MEAN_A1':0.55,
    'SIGMA_A0':0.2,
    'SIGMA_A1':0.2,
    'MIN_A':0,
    'MAX_A':1,
    ### Sex
    'MEAN_S0':0.45,
    'MEAN_S1':0.55,
    'SIGMA_S0':0.2,
    'SIGMA_S1':0.2,
    'MIN_S':0,
    'MAX_S':1,
   ### Noise
    'MEAN_N':0.5,
    'SIGMA_N':0.2,
    'MIN_N':0,
    'MAX_N':1,
    'Weights_proportion':[
        { ### 1: non-differetial taxa
        'N_Species':50,
        'MEAN_WE':0.133,
        'MEAN_WA':0.133,
        'MEAN_WS':0.133,
        'MEAN_WN':0.6,
        'MEAN_WD':0.001,
        },
        { ### 2: differetial taxa
        'N_Species':50,
        'MEAN_WE':0.1,
        'MEAN_WA':0.1,
        'MEAN_WS':0.1,
        'MEAN_WN':0.6,
        'MEAN_WD':0.1,
        },
    ],
    'LE_E':0.08,
    'LE_A':0.08,
    'LE_S':0.08,
}


In [17]:
diff=pd.DataFrame()
for seed in range(0,50):
    metadata= simu_main_meta(config_LE_LA_LS_4,Liner=True)
    microbiome=simu_main_microbiome(metadata,config_LE_LA_LS_4)
    microbiome=microbiome.iloc[:,6:]
    microbiome['Group'] =metadata['Group']
    metadata.index=['s'+str(i) for i in metadata.index]
    microbiome.index=['s'+str(i) for i in microbiome.index]
    pi='config_LE_LA_LS_4'
    params = [('output', 'output_dir', '../simuData/'+pi+'/'+str(seed)+'/'), ('psm', 'caliper', str(0.05)), ('psm', 'ratio', str(1))]
    sample_size, match_drop_unmatched, match, pairs, sum_matched, balance_stats = run_simu_match(data=metadata, target='Group', params=params,features=['Environment', 'Age', 'Sex'],caliper=0.05,ratio=1)
    
    ### difference microbiome
    result = pd.DataFrame()
    res = diff_by_rank_sum(microbiome, target='Group', features=[i for i in microbiome.columns if i!='Group'])
    res.columns = [i+'(raw)' for i in res.columns]
    result = pd.concat([result, res], axis=1, sort=False)
    res = diff_by_signed_rank(microbiome, pairs, features=[i for i in microbiome.columns if i!='Group'])
    res.columns = [i+'(PSM)' for i in res.columns]
    result = pd.concat([result, res], axis=1, sort=False)
    result = result.sort_values(['fdr(PSM)'])
    TrueDiffTax=microbiome.columns[50:100]
    TrueOtherTax=microbiome.columns[0:50]
    DiffTax=result.index[result['p-value(raw)']<0.05]
    OtherTax=microbiome.columns[0:100].difference(DiffTax)
    PSMDiffTax=result.index[result['p-value(PSM)']<0.05]
    PSMOtherTax=microbiome.columns[0:100].difference(PSMDiffTax)
    score=validation_score(DiffTax,PSMDiffTax,TrueDiffTax,OtherTax,PSMOtherTax,TrueOtherTax)
    diff=pd.concat([diff,score])
    save_files(pi,seed,config_LE_LA_LS_4,metadata,microbiome,result)
diff.index=['s'+str(i) for i in range(0,50)]
diff.to_csv('../simuData/LE_LA_LS_changed_4.csv')
diff

,Accuracy,Precision,Recall,F1,psm-Accuracy,psm-Precision,psm-Recall,psm-F1
s0,0.51,0.505051,1.00,0.671141,0.21,0.283582,0.38,0.324786
s1,0.51,0.505051,1.00,0.671141,0.87,1.000000,0.74,0.850575
s2,0.51,0.505051,1.00,0.671141,0.99,1.000000,0.98,0.989899
s3,0.50,0.500000,1.00,0.666667,0.87,0.793651,1.00,0.884956
s4,0.51,0.505051,1.00,0.671141,0.98,1.000000,0.96,0.979592
s5,0.50,0.500000,1.00,0.666667,0.97,0.943396,1.00,0.970874
s6,0.53,0.515789,0.98,0.675862,0.98,1.000000,0.96,0.979592
s7,0.50,0.500000,1.00,0.666667,0.98,0.961538,1.00,0.980392
s8,0.54,0.520833,1.00,0.684932,0.87,1.000000,0.74,0.850575
s9,0.51,0.505051,1.00,0.671141,0.98,1.000000,0.96,0.979592


In [18]:
diff.mean(axis=0)

Accuracy         0.509400
Precision        0.504823
Recall           0.998400
F1               0.670555
psm-Accuracy     0.893000
psm-Precision    0.941220
psm-Recall       0.870400
psm-F1           0.896071
dtype: float64

## 3.5 config(LE)=0.10

In [19]:
config_LE_LA_LS_5 = {
    'N':100,
    ### Environment-drivers
    'MEAN_E0':0.45,
    'MEAN_E1':0.55,
    'SIGMA_E0':0.2,
    'SIGMA_E1':0.2,
    'MIN_E':0,
    'MAX_E':1,
    ### Age
    'MEAN_A0':0.45,
    'MEAN_A1':0.55,
    'SIGMA_A0':0.2,
    'SIGMA_A1':0.2,
    'MIN_A':0,
    'MAX_A':1,
    ### Sex
    'MEAN_S0':0.45,
    'MEAN_S1':0.55,
    'SIGMA_S0':0.2,
    'SIGMA_S1':0.2,
    'MIN_S':0,
    'MAX_S':1,
   ### Noise
    'MEAN_N':0.5,
    'SIGMA_N':0.2,
    'MIN_N':0,
    'MAX_N':1,
    'Weights_proportion':[
        { ### 1: non-differetial taxa
        'N_Species':50,
        'MEAN_WE':0.133,
        'MEAN_WA':0.133,
        'MEAN_WS':0.133,
        'MEAN_WN':0.6,
        'MEAN_WD':0.001,
        },
        { ### 2: differetial taxa
        'N_Species':50,
        'MEAN_WE':0.1,
        'MEAN_WA':0.1,
        'MEAN_WS':0.1,
        'MEAN_WN':0.6,
        'MEAN_WD':0.1,
        },
    ],
    'LE_E':0.1,
    'LE_A':0.1,
    'LE_S':0.1,
}


In [20]:
diff=pd.DataFrame()
for seed in range(0,50):
    metadata= simu_main_meta(config_LE_LA_LS_5,Liner=True)
    microbiome=simu_main_microbiome(metadata,config_LE_LA_LS_5)
    microbiome=microbiome.iloc[:,6:]
    microbiome['Group'] =metadata['Group']
    metadata.index=['s'+str(i) for i in metadata.index]
    microbiome.index=['s'+str(i) for i in microbiome.index]
    pi='config_LE_LA_LS_5'
    params = [('output', 'output_dir', '../simuData/'+pi+'/'+str(seed)+'/'), ('psm', 'caliper', str(0.1)), ('psm', 'ratio', str(1))]
    sample_size, match_drop_unmatched, match, pairs, sum_matched, balance_stats = run_simu_match(data=metadata, target='Group', params=params,features=['Environment', 'Age', 'Sex'],caliper=0.1,ratio=1)
    
    ### difference microbiome
    result = pd.DataFrame()
    res = diff_by_rank_sum(microbiome, target='Group', features=[i for i in microbiome.columns if i!='Group'])
    res.columns = [i+'(raw)' for i in res.columns]
    result = pd.concat([result, res], axis=1, sort=False)
    res = diff_by_signed_rank(microbiome, pairs, features=[i for i in microbiome.columns if i!='Group'])
    res.columns = [i+'(PSM)' for i in res.columns]
    result = pd.concat([result, res], axis=1, sort=False)
    result = result.sort_values(['fdr(PSM)'])
    TrueDiffTax=microbiome.columns[50:100]
    TrueOtherTax=microbiome.columns[0:50]
    DiffTax=result.index[result['p-value(raw)']<0.05]
    OtherTax=microbiome.columns[0:100].difference(DiffTax)
    PSMDiffTax=result.index[result['p-value(PSM)']<0.05]
    PSMOtherTax=microbiome.columns[0:100].difference(PSMDiffTax)
    score=validation_score(DiffTax,PSMDiffTax,TrueDiffTax,OtherTax,PSMOtherTax,TrueOtherTax)
    diff=pd.concat([diff,score])
    save_files(pi,seed,config_LE_LA_LS_5,metadata,microbiome,result)
diff.index=['s'+str(i) for i in range(0,50)]
diff.to_csv('../simuData/LE_LA_LS_changed_5.csv')
diff

,Accuracy,Precision,Recall,F1,psm-Accuracy,psm-Precision,psm-Recall,psm-F1
s0,0.50,0.500000,1.00,0.666667,0.95,0.924528,0.98,0.951456
s1,0.51,0.505051,1.00,0.671141,0.95,0.924528,0.98,0.951456
s2,0.51,0.505051,1.00,0.671141,0.54,0.527027,0.78,0.629032
s3,0.50,0.500000,1.00,0.666667,0.89,1.000000,0.78,0.876404
s4,0.50,0.500000,1.00,0.666667,0.98,1.000000,0.96,0.979592
s5,0.50,0.500000,1.00,0.666667,0.50,0.500000,1.00,0.666667
s6,0.51,0.505051,1.00,0.671141,0.50,0.500000,1.00,0.666667
s7,0.52,0.510204,1.00,0.675676,0.78,1.000000,0.56,0.717949
s8,0.50,0.500000,1.00,0.666667,0.88,1.000000,0.76,0.863636
s9,0.50,0.500000,1.00,0.666667,0.89,1.000000,0.78,0.876404


In [21]:
diff.mean(axis=0)

Accuracy         0.503400
Precision        0.501723
Recall           0.999600
F1               0.668101
psm-Accuracy     0.757400
psm-Precision    0.802862
psm-Recall       0.817200
psm-F1           0.790138
dtype: float64